# CRIM Intervals Demo notebook

This notebook illustrates the use of the new methods `getSoundingCount`, `getMeasure`, and `getTimeSignature`.

In [1]:
%cd ../intervals
from main import *
import pandas as pd
import music21

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


In [2]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei'])
model = corpus.scores[0]
ts = model.getTimeSignature()
ms = model.getMeasure()
sc = model.getSoundingCount()
nr = model.getNoteRest()

Requesting file from https://crimproject.org/mei/CRIM_Model_0008.mei...
Successfully imported.


## `getMeasure` to look at notes and measure

In this example, we build a table of notes and measures, and extract notes from measure 22.

### In the first method, after combining notes and measures, we use dataframe condition to index into the rows with the correct measure

In [3]:
# combine [offsets, notes] and [offsets, measures]
nr_ms = pd.concat([nr, ms], axis=1)
nr_ms

,[Superius],Altus,Tenor,Bassus,[Superius],Altus,Tenor,Bassus
0.0,G4,Rest,Rest,Rest,1.0,1.0,1.0,1.0
4.0,C5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.0,None,Rest,Rest,Rest,2.0,2.0,2.0,2.0
12.0,C5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16.0,D5,G3,Rest,Rest,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...
1256.0,NaN,NaN,D4,G3,NaN,NaN,150.0,150.0
1268.0,C5,G4,NaN,NaN,151.0,151.0,NaN,NaN
1272.0,NaN,NaN,C4,C3,NaN,NaN,151.0,151.0
1284.0,C5,G4,NaN,NaN,152.0,152.0,NaN,NaN


The table has all of the notes and rests offsets, and measures offsets. 

**Unfortunately, we can only see the measure offsets whenever a new measure begins (and we have NaN in other cases). Therefore, we would use forward fill to propagate the measures offsets and give these notes measures.**

In [4]:
# propagate measures offsets forwards (until a new measure begins)
filled_ms = nr_ms.iloc[:, 4:].ffill() 

In [5]:
filled_ms

,[Superius],Altus,Tenor,Bassus
0.0,1.0,1.0,1.0,1.0
4.0,1.0,1.0,1.0,1.0
8.0,2.0,2.0,2.0,2.0
12.0,2.0,2.0,2.0,2.0
16.0,3.0,3.0,3.0,3.0
...,...,...,...,...
1256.0,150.0,150.0,150.0,150.0
1268.0,151.0,151.0,150.0,150.0
1272.0,151.0,151.0,151.0,151.0
1284.0,152.0,152.0,151.0,151.0


**retrieve notes from a specific measure**

In [6]:
# find measure 22 in complete measure table
mask = filled_ms == 22

# retrieve the notes in measure 22
m22_nr = nr_ms.iloc[:, :4][mask].dropna(how='all')

In [7]:
m22_nr

,[Superius],Altus,Tenor,Bassus
168.0,F4,Rest,D4,Rest
172.0,E4,Rest,E4,C3


### We can also turn measures into the notes' index, and then just index directly into the table

Sometimes, the offsets of the same measures can be different accross voices, so doing this isn't extremely safe

In [8]:
# get 1 col of measures (in all voices, measures are expected to have the same offset) (?)
measures = ms.iloc[:, 0]
measures.name = 'Measure'
df = pd.concat([nr, measures], axis=1)
df.Measure.ffill(inplace=True)
# set the "Measure" column as the index
df = df.set_index('Measure')
# get measure slices with .loc, note that .loc slicing is inclusive of the end
mm22_24 = df.loc[22:24, :]

In [9]:
mm22_24

,[Superius],Altus,Tenor,Bassus
Measure,,,,
22.0,F4,Rest,D4,Rest
22.0,E4,Rest,E4,C3
23.0,Rest,C4,C4,None
23.0,NaN,NaN,E4,C3
23.0,NaN,NaN,NaN,C3
23.0,NaN,NaN,D4,NaN
23.0,NaN,NaN,G4,NaN
24.0,Rest,A3,None,D3
24.0,NaN,NaN,F4,NaN


### Important

No matter what we choose to do, we have to remember to use ffil to give all notes measures.

## `getTimeSignture` with notes and measures

### notes with certain time signature

In [10]:
nr_ts = pd.concat([nr, ts], axis=1)
filled_ts = nr_ts.iloc[:, 4:].ffill() 

In [11]:
# retrieve all notes with 
nr_ts.iloc[:, :4][filled_ts == '8/2'].dropna(how='all')

,[Superius],Altus,Tenor,Bassus
1252.0,B4,G4,NaN,NaN
1256.0,NaN,NaN,D4,G3
1268.0,C5,G4,NaN,NaN
1272.0,NaN,NaN,C4,C3
1284.0,C5,G4,NaN,NaN
1288.0,NaN,NaN,C4,C3


### measures with certain time signature

In [12]:
ms_ts = pd.concat([ms, ts], axis=1)
filled_ms_ts = ms_ts.iloc[:, 4:].ffill() 
# filled_ms_ts

In [13]:
# used in the last few measures of the piece
ms_ts.iloc[:, :4][filled_ms_ts == '8/2'].dropna(how='all')

,[Superius],Altus,Tenor,Bassus
1252.0,150.0,150.0,NaN,NaN
1256.0,NaN,NaN,150.0,150.0
1268.0,151.0,151.0,NaN,NaN
1272.0,NaN,NaN,151.0,151.0
1284.0,152.0,152.0,NaN,NaN
1288.0,NaN,NaN,152.0,152.0


In [14]:
# comparing with time signature '3/1' in piece 16 to see 
corpus_16 = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0016.mei'])
model_16 = corpus.scores[0]
ts16 = model.getTimeSignature()
ms16 = model.getMeasure()
ms_ts16 = pd.concat([ms16, ts16], axis=1)
filled_ms_ts16 = ms_ts16.ffill()
# filled_ms_ts16

Requesting file from https://crimproject.org/mei/CRIM_Model_0016.mei...
Successfully imported.


In [22]:
# TODO sometime same offset different measures?
filled_ms_ts16.iloc[:, :4][filled_ms_ts == '3/1'].dropna(how='all')

,[Superius],Altus,Tenor,Bassus
736.0,93.0,93.0,93.0,93.0
744.0,94.0,94.0,93.0,93.0
748.0,94.0,94.0,94.0,94.0
756.0,95.0,95.0,94.0,94.0
760.0,95.0,95.0,95.0,95.0
768.0,96.0,96.0,95.0,95.0
772.0,96.0,96.0,96.0,96.0
780.0,97.0,97.0,96.0,96.0
784.0,97.0,97.0,97.0,97.0
792.0,98.0,98.0,97.0,97.0


## Sounding Count and Cadence

In [16]:
har = model.getHarmonic('d', True, False)
n3 = model.getNgrams(how='modules', df=har, cell_type=str)

In [17]:
#  list of n=3 cadences assuming the interval settings used above
cadences = ['7_Held, 6_-2, 1', '4_Held, 3_-5, 1', '4_Held, 3_4, 1']
# find out which cells satisfy a condition
condition = n3.isin(cadences)
# filter out the cadences using that condition
cadential = n3[condition].dropna(how='all')
# count each type of cadence from any voice pair
cad_frequency = cadential.stack().value_counts()

In [18]:
cadential

,Altus_[Superius],Tenor_Altus,Tenor_[Superius],Bassus_Tenor,Bassus_Altus,Bassus_[Superius]
228.0,NaN,NaN,"7_Held, 6_-2, 1",NaN,NaN,NaN
268.0,"7_Held, 6_-2, 1",NaN,NaN,NaN,NaN,NaN
300.0,NaN,NaN,NaN,"7_Held, 6_-2, 1",NaN,NaN
412.0,NaN,NaN,"7_Held, 6_-2, 1",NaN,NaN,"4_Held, 3_-5, 1"
468.0,"7_Held, 6_-2, 1",NaN,"4_Held, 3_4, 1",NaN,NaN,NaN
508.0,NaN,NaN,NaN,"7_Held, 6_-2, 1",NaN,NaN
604.0,NaN,NaN,"7_Held, 6_-2, 1",NaN,NaN,"4_Held, 3_-5, 1"
732.0,NaN,NaN,NaN,"7_Held, 6_-2, 1",NaN,NaN


In [19]:
cad_frequency

7_Held, 6_-2, 1    8
4_Held, 3_-5, 1    2
4_Held, 3_4, 1     1
dtype: int64

In [20]:
# combine cadential and soundingcount
sc = model.getSoundingCount()
rsc = sc.reindex_like(cadential) # match get sounding count's indices with cadential's indices
cad_sc = pd.concat([cadential, rsc], axis=1) # [offset, cadence] + [offset, sounding]
# how many cadences happen in each voice pair for a given number of sounding voices?
gb_sc = cad_sc.groupby('Sounding').count().T

In [21]:
gb_sc

Sounding,2,3,4
Altus_[Superius],1,1,0
Tenor_Altus,0,0,0
Tenor_[Superius],0,2,2
Bassus_Tenor,1,2,0
Bassus_Altus,0,0,0
Bassus_[Superius],0,1,1
